In [1]:
from pandora_bayesopt.test_functions.pest_control import PestControl

In [2]:
import torch
from botorch.utils.sampling import draw_sobol_samples
from botorch.acquisition import ExpectedImprovement
from pandora_bayesopt.acquisition.gittins import GittinsIndex
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer

In [3]:
# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [4]:
seed = 42
torch.manual_seed(seed)

In [5]:
dim = 25
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

In [6]:
def objective_function(X):
    choice_X = torch.floor(5*X)
    choice_X[choice_X == 5] = 4
    return PestControl(negate=True)(choice_X)

In [7]:
init_y = objective_function(init_x)
print(init_y)

tensor([-18.2056, -20.6544, -21.0508, -18.6940, -18.3764, -17.7940, -19.4588,
        -16.9936, -18.8900, -18.1152, -20.0720, -17.6120, -20.1732, -17.1396,
        -20.2652, -17.8624, -18.2528, -17.3620, -18.8732, -19.9800, -18.9572,
        -18.5348, -19.2760, -18.3476, -17.5500, -18.4972, -18.3236, -18.3292,
        -18.8908, -18.4700, -18.7872, -19.4580, -18.6132, -18.8760, -19.4020,
        -19.0400, -16.8056, -17.1168, -19.3272, -16.9424, -19.4556, -18.5840,
        -18.1244, -19.0356, -17.9636, -18.8448, -17.9572, -19.3644, -18.8120,
        -18.4712, -18.0804, -17.4176])


In [8]:
Optimizer = BayesianOptimizer(
        objective=objective_function, 
        dim=dim, 
        maximize=True, 
        initial_points=init_x,
        input_standardize=True
    )

Optimizer.run(
    num_iterations=2, 
    acquisition_function_class=ExpectedImprovement
)

Iteration 0, New point: [0.32133795 0.81804    0.76476365 0.93815855 0.61110898 0.57763049
 0.09588868 0.44570932 0.80624984 0.85456994 0.92152701 0.7642226
 0.95151181 0.77668428 0.47258786 0.48081227 0.77926163 0.23886343
 0.49950168 0.98136905 0.37207303 0.6903533  0.14503833 0.23928996
 0.15685175], New value: -16.749599408475916
Best observed value: -16.749599408475916
Current acquisition value: tensor(0.0949)
Cumulative cost: 1.0

Iteration 1, New point: [0.25075209 0.77800755 0.80366839 0.95705485 0.58745853 0.57157827
 0.08219699 0.44852147 0.83344093 0.86296361 0.9949403  0.77584257
 0.93852888 0.74063941 0.46319227 0.45482402 0.78568235 0.23154683
 0.52072701 0.98996376 0.38904766 0.64769197 0.08612052 0.22073918
 0.16727214], New value: -16.946399756321362
Best observed value: -16.749599408475916
Current acquisition value: tensor(0.1027)
Cumulative cost: 2.0



In [9]:
best_history = Optimizer.get_best_history()
best_history

[-16.80559940323234, -16.749599408475916, -16.749599408475916]